# Movie-Lens 100k Dataset

## Data Import

In [2]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/han942/vscode/refs/heads/main/datafile/Recsys/ml-100k/u.data',sep='\t',
                   names=['userID','movieID','ratings','timestamp'])
data.head()

,userID,movieID,ratings,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
#각 index가 ID가 됨.
items = pd.read_csv('https://raw.githubusercontent.com/han942/vscode/refs/heads/main/datafile/Recsys/ml-100k/u.item',encoding='latin-1',
                    sep='|',header=None,names=['moviename','launching','no','image','0','1','2','3','4','5','6','7',
                                               '8','9','10','11','12','13','14','15','16','17','18'])
items.head()

,moviename,launching,no,image,0,1,2,3,4,5,...,9,10,11,12,13,14,15,16,17,18
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
users = pd.read_csv('https://raw.githubusercontent.com/han942/vscode/refs/heads/main/datafile/Recsys/ml-100k/u.user',sep='|',
                   names=['userID','age','gender','occupation','zipcode'])
users.head()

,userID,age,gender,occupation,zipcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


## Content Based Filtering

In [5]:
data

,userID,movieID,ratings,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
